## Yelp

### reviews

In [1]:
import pandas as pd

In [2]:
reviews = pd.read_csv("d:\\Downloads\\bq-results-20241021-182102-1729534874175.csv")

In [3]:
# Pasamos la columna date a formato date
reviews['date'] = pd.to_datetime(reviews['date'], format='%Y-%m-%d %H:%M:%S UTC')


In [4]:
# cool va de 0 a 72
# stars va 2 a 5
# funny de 0 a 35
# useful de 0 a 324
reviews.describe()

,cool,stars,date,funny,useful
count,40000.000000,40000.00000,40000,40000.000000,40000.000000
mean,0.406900,3.89025,2016-10-07 06:43:31.074099968,0.269475,0.830550
min,0.000000,2.00000,2005-05-18 00:28:34,0.000000,0.000000
25%,0.000000,3.00000,2014-09-13 09:08:40.500000,0.000000,0.000000
50%,0.000000,4.00000,2017-03-06 00:44:27.500000,0.000000,0.000000
75%,0.000000,5.00000,2019-03-10 02:13:01.249999872,0.000000,1.000000
max,72.000000,5.00000,2022-01-19 17:18:47,35.000000,324.000000
std,1.587371,1.01224,NaN,1.161052,2.721493


### bussines


In [5]:
import pandas as pd

# Cargar el archivo .pkl
df = pd.read_pickle("D:\\Desktop\\proyecto_grupal\\business.pkl")

In [6]:
# Borrar columnas duplicadas
duplicated_columns = df.columns[df.columns.duplicated()]

# Eliminar columnas duplicadas
df = df.loc[:, ~df.columns.duplicated()]

In [7]:
# Filtramos por florida
df_business= df[df["state"].isin(["FL"])]


In [8]:
# Filtramos por restaurantes
df_business = df_business[df_business["categories"].str.contains(r"(?i)\brestaurants\b", na=False)] 

In [9]:
# Reemplazamos los nan por desconocidos para no tener problemas en el modelo de machine learning
df_business["address"] = df_business["address"].fillna('Desconocido')
df_business["postal_code"] = df_business["postal_code"].fillna('Desconocido')

In [10]:
# Hacemos un nuevo dataframe de la columna desanidada attributes
from pandas import json_normalize
df_attributes = json_normalize(df_business['attributes'])

In [11]:
# Borramos columnas que no necesitamos 
df_business.drop(columns=['latitude', 'longitude',
       'attributes', 'hours'],inplace= True)

In [12]:
# Guardar como archivo CSV
df_business.to_csv("df_business_limpio.csv", index=False)
df_attributes.to_csv("df_attributes.csv")

Los 0 son los restaurantes que no tienen el servicio y los 1 son los que si tienen el servicio

In [13]:
# Reemplazar True por 1 y False por 0
df_attributes.replace({True: 1, False: 0}, inplace=True)

# Reemplazar los NaN por 0
df_attributes.fillna(0, inplace=True)

In [14]:
# Reemplazar u' por nada en todas las columnas de tipo string
df_attributes = df_attributes.applymap(lambda x: x.replace("u'", "").replace("'", "") if isinstance(x, str) else x)

# Reemplazar True por 1 y False por 0
df_attributes["WiFi"].replace({"free": 1, "no": 0}, inplace=True)


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_3396\3663465635.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_attributes = df_attributes.applymap(lambda x: x.replace("u'", "").replace("'", "") if isinstance(x, str) else x)


### Join entre df_business_limpio y df_reviews

reviews tiene 40mil registros

In [15]:
df_yelp = pd.merge(df_business,reviews,on="business_id",how="inner")

In [16]:
df_yelp.head(10)

,business_id,name,address,city,state,postal_code,stars_x,review_count,is_open,categories,text,cool,stars_y,date,funny,review_id,useful,user_id
0,seKihQKpGGnCeLuELRQPSQ,Twin Peaks,6880 E 82nd St,Indianapolis,FL,46250,3.5,257,1,"Sports Bars, American (New), American (Traditi...",Good was on par with your typical sports bar. ...,0,3,2015-01-11 22:49:33,0,CSebyfJjT740KKgE6IPYcg,0,ZYNEE_VRshepHB599zUWcA
1,seKihQKpGGnCeLuELRQPSQ,Twin Peaks,6880 E 82nd St,Indianapolis,FL,46250,3.5,257,1,"Sports Bars, American (New), American (Traditi...","With chain restaurants, I write short reviews ...",0,4,2019-07-08 00:18:19,3,B5aECxALkQZgQQQ4oQiVSA,0,_TKXaTPT2Yu0rRFqYzs44g
2,seKihQKpGGnCeLuELRQPSQ,Twin Peaks,6880 E 82nd St,Indianapolis,FL,46250,3.5,257,1,"Sports Bars, American (New), American (Traditi...",Been wanting to come to twin peaks for a while...,0,5,2022-01-18 20:36:24,0,uOzC-IXnlc5hJ2h-0jNRKQ,0,ZphdnOBaO5Qd3STxXjspVw
3,ppFCk9aQkM338Rgwpl2F5A,Wawa,3604 Chestnut St,Philadelphia,FL,19104,3.0,56,1,"Restaurants, Automotive, Delis, Gas Stations, ...",This place is pretty well-run - it's always cl...,0,4,2009-01-25 04:08:52,0,PUNdXZqu6PvqhOWnsqIkAw,1,OZTZgCcxO7fcFkZ3LYDroA
4,uDQgmudYDKiB6n4vwbEeDg,Chrysalis,9040 Carothers Pkwy,Franklin,FL,37067,4.5,112,1,"Restaurants, Italian, Nightlife, Bars, Cocktai...","We ordered take-out from this new place, and s...",1,4,2021-02-15 19:44:15,1,QhOnn7yj_oheUXjzAsT6ug,4,Z18Bze20mEVqRXAfe8DjZg
5,Edg22x3CZkIv0GUib2oEFA,Mellow Mushroom,8227 Oak St,New Orleans,FL,70118,3.5,149,0,"Pizza, Restaurants, Bars, Nightlife, Sandwiches",Fun ambiance and food is good not great. Servi...,0,3,2019-09-14 22:43:20,0,YMpRLhAt7r2FYVp0JSpfdw,0,9lAp1eOFlvmA21pvLn6UsQ
6,Edg22x3CZkIv0GUib2oEFA,Mellow Mushroom,8227 Oak St,New Orleans,FL,70118,3.5,149,0,"Pizza, Restaurants, Bars, Nightlife, Sandwiches",The good. We had the pizza. Delicious crust. C...,0,3,2016-03-25 03:10:54,0,mNgCyN6iuDHrJFqLwEBqXQ,0,NEgCqpLI1QCN5TxKn23fiA
7,W57cR9a7XP6RX56MSWHJJA,Las Margaritas,514 S Rampart St,New Orleans,FL,70163,3.5,58,0,"Mexican, Restaurants",Fast and friendly service. My grandson said t...,0,3,2016-06-19 00:57:10,0,c1DeVcZ58Vmc7m75rSj1zQ,0,-ya_NmklzNvBXQs8mdiwfw
8,-7GDqSUaXrpC8Ql7nDBxWA,B2 Bluefin,401 E City Ave,Bala Cynwyd,FL,19004,4.0,92,0,"Nightlife, Bars, Cocktail Bars, Japanese, Rest...",Great atmosphere. Delicious food. Weird servic...,0,4,2018-06-23 00:55:43,0,ZRf8qZM1Rq21e5cI3HIcpA,0,EsMMxTs0arl-c0a4ZuYtsg
9,XwoXi0M66RU6Gouq-DSDHw,G Peppers Grill & Tavern,8745 Gunn Hwy,Odessa,FL,33556,4.0,82,1,"Restaurants, Mexican, American (Traditional)","Starting Thursday, 12.15.11\nCOWBOY QUESADILLA...",0,5,2011-12-12 14:07:48,0,Rx4XxoA2U4LFCkGjoJqPjA,0,Mxb3qiFonMIoFzAUPThRmQ


## Google Maps

### Reviews estados Florida

Review florida

In [17]:
import json
import pandas as pd
import glob
from pandas import json_normalize

In [18]:

# Ruta a los archivos JSON
ruta_archivos = "D:\\Downloads\\review-Florida\\*.json"

# Inicializar una lista para almacenar los datos
datos = []

# Leer cada archivo JSON
for archivo in glob.glob(ruta_archivos):
    with open(archivo, "r") as f:
        for linea in f:
            objeto = json.loads(linea)
            datos.append(objeto)

# Crear un DataFrame con todos los datos
reviews_florida = pd.DataFrame(datos)

In [20]:
reviews_florida

,user_id,name,time,rating,text,pics,resp,gmap_id
0,101471856155148729010,Julie A. Gerber,1628003250740,1,Update: Their “reply” to my review amounted to...,None,"{'time': 1627042799532, 'text': 'Thank you for...",0x8893863ea87bd5dd:0x9383ebf973e74abb
1,115477234789038326051,Martin Sheffield,1595031217005,5,He's a knowledgeable doctor but the way he run...,None,"{'time': 1582464056733, 'text': 'Thank you for...",0x8893863ea87bd5dd:0x9383ebf973e74abb
2,101805010244892834381,Brian Truett,1522924253567,5,"Best doctor I've ever had, I never wait to be ...",None,None,0x8893863ea87bd5dd:0x9383ebf973e74abb
3,106344422881493743981,Tina Sun,1467907819586,1,I was told he is a good doctor. I was trying t...,None,None,0x8893863ea87bd5dd:0x9383ebf973e74abb
4,100875113069561776529,James Haynes,1480683415081,5,Takes the time to actually get to know his pat...,None,None,0x8893863ea87bd5dd:0x9383ebf973e74abb
...,...,...,...,...,...,...,...,...
749995,103116157833516568554,Carmen Roman,1605310910233,5,None,None,None,0x88d8df729ca5eda7:0x8131504abdbfe7fe
749996,101308524654469917198,Linda Payette,1521892525593,5,None,None,None,0x88d8df729ca5eda7:0x8131504abdbfe7fe
749997,112274337653160939064,elmer Garcia,1541205269300,4,None,None,None,0x88d8df729ca5eda7:0x8131504abdbfe7fe
749998,106993940912678181321,Gerald Zehme,1529192818272,5,None,None,None,0x88d8df729ca5eda7:0x8131504abdbfe7fe


In [21]:
# Borramos columnas que no necesitamos
reviews_florida.drop(columns=[ 'time', 'pics','resp'],inplace=True)

In [22]:
# Reemplazamos los Nan por "Sin Nombre" y "Sin Comentario"
reviews_florida["name"] = reviews_florida["name"].fillna('Sin Nombre')
reviews_florida["text"] = reviews_florida["text"].fillna('Sin Comentario')

### metadata-sitios

In [68]:
# Ruta a los archivos JSON
ruta_archivos = "D:\\Downloads\\metadata-sitios\\*.json"

# Inicializar una lista para almacenar los datos
datos = []

# Leer cada archivo JSON
for archivo in glob.glob(ruta_archivos):
    with open(archivo, "r") as f:
        for linea in f:
            objeto = json.loads(linea)
            datos.append(objeto)

# Crear un DataFrame con todos los datos
df_sitios = pd.DataFrame(datos)

ERROR! Session/line number was not unique in database. History logging moved to new session 427


In [101]:
df_sitios.drop(columns=['Unnamed: 0', 'description', 'latitude','longitude','hours', 'state', 'relative_results', 'url',"MISC"],inplace=True)

In [116]:
df_sitios["address"]= df_sitios["address"].fillna("Sin registro")
df_sitios["price"]= df_sitios["price"].fillna("Sin registro")
df_sitios["MISC"]= df_sitios["MISC"].fillna("Sin registro")

In [72]:
# Renombramos los nulos de la columna "category"
df_sitios['category'] = df_sitios['category'].fillna('')

# Filtramos bussines que sean restaurantes
df_sitios = df_sitios[df_sitios['category'].astype(str).str.contains('restaurant', case=False)]

### Hacemos un join con df_sitios y df_florida

In [74]:
# Realizar el join
df_google = pd.merge(df_sitios, reviews_florida, on='gmap_id', how='inner')

In [76]:
df_google.rename(columns={"name_x": "User", "name_y": "restaurant"}, inplace=True)


In [78]:
df_google.to_csv("df_google.csv")

## Modelo de machine learning

In [119]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [122]:
# Limpiar datos: eliminar filas con texto nulo o vacío
df = df.dropna(subset=['text'])  # Eliminar filas donde 'text' es None
df = df[df['text'].str.strip() != '']  # Eliminar filas donde 'text' está vací

In [123]:
# Vectorizar el texto
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['text'])


In [124]:
# Calcular la matriz de similitud
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [125]:
def recommend_similar_restaurants(restaurant_name, cosine_sim=cosine_sim):
    # Obtén el índice del restaurante que coincide con el nombre
    idx = df[df['Name_restaurant'] == restaurant_name].index[0]
    
    # Obtén las puntuaciones de similitud de todas las filas con ese restaurante
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Ordena las restaurantes basados en la similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Obtener los 5 restaurantes más similares
    sim_scores = sim_scores[1:6]  # Excluyendo el mismo restaurante
    
    # Obtén los índices de los restaurantes
    restaurant_indices = [i[0] for i in sim_scores]
    
    # Retorna los restaurantes similares
    return df.iloc[restaurant_indices]

In [137]:
df["Name_restaurant"][2000]

'Alley Cat Seafood, Beer House & Wine Boutique'

In [138]:
# Prueba la función
recommended_restaurants = recommend_similar_restaurants("Alley Cat Seafood, Beer House & Wine Boutique")


In [139]:
recommended_restaurants["Name_restaurant"].value_counts()

Name_restaurant
Maoz Vegetarian               1
Las Delicias De Clara         1
Zócalo Folsom                 1
Green Life Market             1
Norkys Peruvian Restaurant    1
Name: count, dtype: int64